In [ ]:
library(DESeq2)
library(DEGreport)
library(tidyverse)
library(patchwork)
library(gplots)
library(EnhancedVolcano)

In [ ]:
expdesign <- read.table("./source_data/reference/mfa_samplelist.txt",row.names = 1)
readcount <- read.table('./source_data/batch_correction/batch_cor_GeneCount_mfa.txt',header=T)

rownames(readcount) <- readcount$gene_chr
readcount <- readcount[,c(2:25)]

In [ ]:
for(i in list('G1-G2','G2-G3','G3-G4')){
    if(i == 'G1-G2'){rcount <- readcount[,c(1,2,3,4,5,6,7,8,9,10,11,12)]
                     edesign <- expdesign[c(1,2,9,10,17,18,3,4,11,12,19,20),]
                     edesign$condition =  factor(c(rep('C1',6),rep('C2',6)))} 
    else if(i == 'G2-G3'){rcount <- readcount[,c(7,8,9,10,11,12,13,14,15,16,17,18)]
                          edesign <- expdesign[c(3,4,11,12,19,20,5,6,13,14,21,22),]
                          edesign$condition =  factor(c(rep('C2',6),rep('C3',6)))}
    else {rcount <- readcount[,c(13,14,15,16,17,18,19,20,21,22,23,24)]
          edesign <- expdesign[c(5,6,13,14,21,22,7,8,15,16,23,24),]
          edesign$condition =  factor(c(rep('C3',6),rep('C4',6)))}
                                       
    dds <- DESeqDataSetFromMatrix(countData = rcount,
                                  colData = edesign,
                                  design = ~ condition)
                                       
                                       
    ################
    # DEG analysis #
    ################
    dds <- DESeq(dds)
    vsd <- vst(dds, blind=FALSE)
    res <- results(dds)
    
    res$gene <- row.names(res)
    split_result = str_split(res$gene,"_", 2)
    res$gene = lapply(X = split_result, FUN = "[", 1)
    res$gene <- as.character(res$gene)
                                       
                                       
    ##################################
    #######  data parsing  ###########
    ##################################
    print(nrow(res))
    DEG_result <- res[!grepl("^LOC", res$gene),]
    DEG_result <- DEG_result[,c(7,2,6)]
    print(nrow(DEG_result))
    DEG_result <- na.omit(DEG_result)
    print(nrow(DEG_result))
    up <- DEG_result[as.numeric(DEG_result$padj)< 0.05 & as.numeric(DEG_result$log2FoldChange)> 1,] 
    down <- DEG_result[as.numeric(DEG_result$padj)< 0.05 & as.numeric(DEG_result$log2FoldChange)< -1,] 
    
    print(nrow(up))
    print(nrow(down))

    write.table(up, file=paste0('./source_data/DEmRNA/',i,'_upDEG.txt'), sep='\t')
    write.table(down, file=paste0('./source_data/DEmRNA/',i,'_downDEG.txt'), sep='\t')
                                       
                                       
    #######################################
    #######     Volcano Plot    ###########
    #######################################
    data <- res
    data <- data[!grepl("^LOC", data$gene),]
    rownames(data) <- data$gene
    keyvals <- ifelse(data$padj < 0.05 & data$log2FoldChange < -1, 'blue',
                      ifelse(data$padj < 0.05 & data$log2FoldChange > 1, 'red','grey'))
    
    keyvals[is.na(keyvals)] <- 'grey'
    names(keyvals)[keyvals == 'red'] <- 'UP'
    names(keyvals)[keyvals == 'grey'] <- 'Not sig'
    names(keyvals)[keyvals == 'blue'] <- 'DOWN'
    
    plot = EnhancedVolcano(data,lab=rownames(data),
                            x='log2FoldChange',
                            y='padj',
                            selectLab = rownames(data)[which(names(keyvals) %in% c('UP', 'DOWN'))],
                            xlim = c(-2.5,2.5),
                            ylim = c(0,20),
                            pCutoff = 0.05,
                            FCcutoff = 1,
                            pointSize = 2.0,
                            labSize = 3.5,
                            colCustom = keyvals,
                            colAlpha = 0.5,
                            legendPosition = 'left',
                            legendLabSize = 15,
                            legendIconSize = 5.0,
                            gridlines.major = FALSE,
                            gridlines.minor = FALSE,
                            border = 'partial',
                            borderWidth = 0.5,drawConnectors=TRUE,
                            borderColour = 'black',
                            title = i,)
    print(plot)
}